In [ ]:
import glob
import json
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import re



In [ ]:
# thank you chatgpt <3
def parse_volume_string(input_string):
    pattern = r'(\d+(\.\d+)?)\s*(\w+)\s+(\d+(\.\d+)?)\s*(\w+)\s*(.*)'
    matches = re.search(pattern, input_string)
    if matches:
        amount1 = matches.group(1)
        unit1 = matches.group(3)
        amount2 = matches.group(4)
        unit2 = matches.group(6)
        trailing_text = matches.group(7).strip()
        return amount1, unit1, amount2, unit2, trailing_text
    return None

def parse_single_volume(input_string):
    pattern = r'\s*(\d+(\.\d*)?|\.\d+)\s*(\w+)\s*'
    matches = re.match(pattern, input_string)
    if matches:
        amount = matches.group(1)
        unit = matches.group(3)
        return amount, unit
    return None

In [ ]:
product_files = glob.glob("data/products/*")
products = []
for product_file in product_files:
    with open(product_file) as file:
        products.append(pd.json_normalize(json.loads(file.read())))
        
df_products = pd.concat(products, axis=0)
df_products = df_products[df_products['product_name'].notnull()]
df_products = df_products.reset_index().rename(columns={'index':'internal_product_id'})

## product ratings
df_products['rating'] = df_products['rating'].str.replace("width:","")
df_products['rating'] = df_products['rating'].str.replace("%","")
df_products['rating'] = df_products['rating'].astype(float)
df_products['rating'] = df_products['rating']/100 * 5

df_products['n_loves'] = df_products['n_loves'].str.replace(".","")
df_products['n_loves'] = df_products['n_loves'].str.replace("K","00")
df_products['n_loves'] = df_products['n_loves'].str.replace("M","000000")
df_products['n_loves'] = df_products['n_loves'].astype(float)

product_options = []
for product in df_products.iterrows():
    url = product[1]['url']
    df_options = pd.json_normalize(product[1]['options'])
    df_options['url'] = url
    product_options.append(df_options)
    
df_products = df_products.merge(pd.concat(product_options), how='left', on='url')

df_products['name'] =  df_products['name'].str.lower()
df_products['size'] =  df_products['size'].str.lower()


df_products['price'] = df_products['price'].str.replace("$","")
df_products['price'] = df_products['price'].astype(float)

df_products['out_of_stock'] = False
df_products['out_of_stock'] = df_products['name'].str.contains('out of stock')

df_products['limited_edition'] = False
df_products['limited_edition'] = df_products['name'].str.contains('limited edition')

df_products['new_product'] = False
df_products['new_product'] = df_products['name'].str.contains('new')

df_products['few_left'] = False
df_products['few_left'] = df_products['name'].str.contains('only a few left')

df_products['sale'] = False
df_products['sale'] = df_products['name'].str.contains('sale')

df_products['refill'] = False
df_products['refill'] = df_products['name'].str.contains('refill')

df_products.loc[df_products['size'].isna(), 'size'] = df_products['name']
df_products.loc[df_products['size']==df_products['name'],'name'] = None


df_products['size'] = df_products['size'].str.replace("out of stock","")
df_products['size'] = df_products['size'].str.replace("limited edition","")
df_products['size'] = df_products['size'].str.replace("new","")
df_products['size'] = df_products['size'].str.replace("only a few left","")
df_products['size'] = df_products['size'].str.replace("sale","")
df_products['size'] = df_products['size'].str.replace("size","")
df_products['size'] = df_products['size'].str.replace("refill","")
df_products['size'] = df_products['size'].str.replace("color","")
df_products['size'] = df_products['size'].str.replace(":","")
df_products['size'] = df_products['size'].str.replace("-","")
df_products['size'] = df_products['size'].str.replace("mini","")
df_products['size'] = df_products['size'].str.replace("fl oz","floz")
df_products['size'] = df_products['size'].str.replace("fl. oz","floz")
df_products['size'] = df_products['size'].str.replace("oz.","oz ")
df_products['size'] = df_products['size'].str.replace("/"," ")

df_products['size'] = df_products['size'].replace(r'\s+', ' ', regex=True)


df_products['sku'] = df_products['sku'].str.replace("Item ","")

df_products['size'] = df_products['size'].fillna("")

df_products['amount_a'], df_products['unit_a'], df_products['amount_b'], df_products['unit_b'], df_products['misc_info'] = df_products['size'].apply(parse_string).str
df_products[['amount_a','amount_b']] = df_products[['amount_a','amount_b']].astype(float)

df_products['amount_single'], df_products['unit_single'] = df_products[df_products['amount_a'].isna()]['size'].apply(parse_single_volume).str

In [ ]:
# df_products[(df_products['amount_a'].isna()) & (df_products['amount_single'].isna())]['size'].value_counts()[0:100]